### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, IsolationForest
from sklearn.metrics import  f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import seaborn as sns
import plotly.express as px
import sweetviz as sv
import pickle
from boruta import BorutaPy
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVR

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

### Helper Functions

In [2]:
alvo='limite_adicional'

def simple_model_test(model, nome):
    #model definition

    # model fit
    model.fit(X_train, y_train)

    #model predict
    y_hat = model.predict(X_test)
    
    # evaluate
    f1 = np.round(f1_score( y_test_, y_hat_ ), 2)
    
    print('{}\n f1: {}'.format(model_name, f1))
    return f1

def send_model(model):
     # model fit
    model.fit(X_train[cols_selected], y_train)
    pickle.dump(model, open('../parameters/model.pkl', 'wb'))
    print('Model submited')
    return None

def target_encoding(df, column):
    target = df.groupby(column)[alvo].mean()
    df[column] = df[column].map(target)
    pickle.dump(target, open(f'../parameters/{column}_encode.pkl', 'wb'))
    print(f'Target Encode to {column}')
    
def frequency_encoding(df, column):
    frequency = df.groupby(column)[alvo].count() / len(df)
    df[column] = df[column].map(frequency)
    pickle.dump(target, open(f'../parameters/{column}_encode.pkl', 'wb'))
    print(f'Frequency Encode to {column}')
    

In [4]:
def encodes_obrigatorios(df, teste=True):
    num_cols = df.select_dtypes('number').drop('id_cliente', axis=1).columns


    map_bool = {'Sim': 1, 'Não': 0}
    map_alvo = {'Conceder': 1, 'Negar': 0}
    string_cols = ['investe_exterior', 'pessoa_polit_exp']

    df['investe_exterior'] = df['investe_exterior'].map(map_bool)
    df['pessoa_polit_exp'] = df['pessoa_polit_exp'].map(map_bool)
    
    if teste==False:
        df[alvo] = df[alvo].map(map_alvo)
    return df

### Load Data 

In [10]:
path = '../data/train.csv'

df_raw = pd.read_csv(path)
df_raw=encodes_obrigatorios(df_raw, teste=False)

df_test = pd.read_csv('../data/test.csv')
df_test=encodes_obrigatorios(df_test, teste=True)

### Data Description

In [11]:
df1 = df_raw.copy()

#### Nulls

In [12]:
df1.isna().sum()

id_cliente                 0
idade                      0
saldo_atual                0
divida_atual               0
renda_anual                0
valor_em_investimentos     0
taxa_utilizacao_credito    0
num_emprestimos            0
num_contas_bancarias       0
num_cartoes_credito        0
dias_atraso_dt_venc        0
num_pgtos_atrasados        0
num_consultas_credito      0
taxa_juros                 0
investe_exterior           0
pessoa_polit_exp           0
limite_adicional           0
dtype: int64

#### Dtypes

In [13]:
df1.dtypes

id_cliente                   int64
idade                        int64
saldo_atual                float64
divida_atual               float64
renda_anual                float64
valor_em_investimentos     float64
taxa_utilizacao_credito    float64
num_emprestimos              int64
num_contas_bancarias         int64
num_cartoes_credito          int64
dias_atraso_dt_venc          int64
num_pgtos_atrasados          int64
num_consultas_credito        int64
taxa_juros                   int64
investe_exterior             int64
pessoa_polit_exp             int64
limite_adicional             int64
dtype: object

### Outliers inspection

In [147]:
df1['pessoa_polit_exp'].value_counts()

0    8917
1     583
Name: pessoa_polit_exp, dtype: int64

In [145]:
df1['limite_adicional'].value_counts()

0    7995
1    1505
Name: limite_adicional, dtype: int64

In [14]:
dashboard=sv.analyze(df1, target_feat='limite_adicional')

                                             |          | [  0%]   00:00 -> (? left)

In [15]:
dashboard.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### Data Filtering

In [16]:
df2 = df1.copy()

### Feature Engineering

In [191]:
df3 = df2.copy()
df3['comb_exterior_politic'] = df3['investe_exterior'] + df3['pessoa_polit_exp']
df3 = df3.drop(['investe_exterior','pessoa_polit_exp'], axis=1)

### Colum Selection

In [192]:
df4 = df3.copy()

### Data Preparation

In [193]:
df5 = df4.copy()

In [194]:
variables_robust=['idade','taxa_juros','num_consultas_credito',"num_pgtos_atrasados","num_cartoes_credito","num_contas_bancarias"]
num_cols = df5.select_dtypes('number').drop(['id_cliente','limite_adicional'], axis=1).columns

#rs_idade=pp.RobustScaler(quantile_range=(0, 95.0))
mms = pp.MinMaxScaler()

df5[variables_robust]=rs_idade.fit_transform(df5[variables_robust])    
df5[num_cols] = mms.fit_transform(df5[num_cols])

for i in [rs_idade,mms]:
    pickle.dump(i,open(f"{i}.pkl","wb"))

In [186]:
def data_preparation(df, teste=True):
    #df[variables_robust]=rs_idade.transform(df[variables_robust])
    df[num_cols]=mms.transform(df[num_cols])
    return df

### Model Train

In [187]:
X = df5.drop(['id_cliente', alvo], axis=1)
y = df5[alvo]

In [188]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)


100%|██████████| 29/29 [00:11<00:00,  2.53it/s]


In [189]:
models.sort_values('F1 Score', ascending=False)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.87,0.72,0.72,0.87,0.33
RandomForestClassifier,0.88,0.70,0.70,0.87,1.06
XGBClassifier,0.87,0.71,0.71,0.86,0.58
BaggingClassifier,0.87,0.70,0.70,0.86,0.40
AdaBoostClassifier,0.87,0.69,0.69,0.86,0.54
ExtraTreesClassifier,0.86,0.64,0.64,0.84,0.80
DecisionTreeClassifier,0.84,0.70,0.70,0.84,0.09
ExtraTreeClassifier,0.82,0.66,0.66,0.82,0.05
LogisticRegression,0.84,0.57,0.57,0.81,0.07


In [ ]:
fpnfo

In [172]:
pipe = clf.models['RandomForestClassifier']
pred = pipe.predict(X_test)

f1_score(y_test, pred, average='micro')

0.8778947368421053

In [173]:
types_f1 = ['micro', 'macro', 'samples', 'weighted', 'binary']
for t in types_f1:
    try:
        print(t,f1_score(y_test, pred, average=t))
    except ValueError:
        print(f'Not possible {t}')

micro 0.8778947368421053
macro 0.7342897231559322
Not possible samples
weighted 0.8673669924252495
binary 0.5389507154213037


### Teste

In [174]:
X_submission=df_test.copy()
X_submission = df_test.drop('id_cliente', axis =1)
X_submission= data_preparation(X_submission)
X_submission[variables_robust]=rs_idade.fit_transform(X_submission[variables_robust])    
X_submission[num_cols] = mms.fit_transform(X_submission[num_cols])
ids = df_test['id_cliente']

In [175]:
pred = pipe.predict(X_submission)

In [176]:
submission = pd.DataFrame()

In [177]:
submission['id_cliente'] = ids
submission['limite_adicional'] = pred

In [178]:
map_resposta = {0: 'Negar', 1: 'Conceder'}

In [179]:
submission['limite_adicional'] = submission['limite_adicional'].map(map_resposta)

In [180]:
submission.to_csv('../data/submissao.csv', index=False)

In [181]:
pd.read_csv('../data/submissao.csv')

,id_cliente,limite_adicional
0,2,Conceder
1,5,Negar
2,6,Negar
3,8,Negar
4,10,Conceder
...,...,...
2995,12484,Negar
2996,12487,Negar
2997,12489,Negar
2998,12495,Conceder
